In [1]:
from collections import namedtuple
from decimal import Decimal
import inspect
from pathlib import Path

import pandas as pd

#import arrow

from django.contrib.auth import get_user_model
from django.db import connection, connections, models, reset_queries


def show_class(class_):
    for f in class_._meta.fields:
        print(f.name)


def show_obj(obj):
    for f in obj._meta.model._meta.fields:
        print('{} --> {}'.format(f.name, getattr(obj, f.name)))
        

def query_count():
    count = sum(len(c.queries) for c in connections.all())
    reset_queries()
    return count

def show_func_code(f):
    print(inspect.getsource(f))

%load_ext autoreload
%autoreload 2

In [4]:
food_df = pd.DataFrame.from_records(Foods.objects.all().values())
food_group_df = pd.DataFrame.from_records(FoodGroup.objects.all().values())
food_type_df = pd.DataFrame.from_records(FoodType.objects.all().values())
food_notes_df = pd.DataFrame.from_records(FoodNotes.objects.all().values())

ing_df = pd.DataFrame.from_records(Ingredients.objects.all().values())
srv_df = pd.DataFrame.from_records(Servings.objects.all().values())

macros_df = pd.DataFrame.from_records(Macros.objects.all().values())


food_df.to_csv(data_dir / 'food.csv', index=False)
food_group_df.to_csv(data_dir / 'food_group.csv', index=False)
food_type_df.to_csv(data_dir / 'food_type.csv', index=False)
food_notes_df.to_csv(data_dir / 'food_notes.csv', index=False)
ing_df.to_csv(data_dir / 'ing.csv', index=False)
srv_df.to_csv(data_dir / 'srv.csv', index=False)
macros_df.to_csv(data_dir / 'macros.csv', index=False)


In [3]:
food_df = pd.read_csv(Path.home() / 'data/food.csv')
food_group_df = pd.read_csv(Path.home() / 'data/food_group.csv')
food_type_df = pd.read_csv(Path.home() / 'data/food_type.csv')
#food_notes_df = pd.read_csv(Path.home() / 'data/food_notes.csv')
ing_df = pd.read_csv(Path.home() / 'data/ing.csv')
srv_df = pd.read_csv(Path.home() / 'data/srv.csv')
#macros_df = pd.read_csv(Path.home() / 'data/macros.csv')




In [4]:
usda = MacroUser.objects.get(id=2)
su_paul = MacroUser.objects.get(id=1)

food_df.loc[food_df.user_id == 1, 'user_id'] = usda.id
food_df.loc[food_df.user_id == 4, 'user_id'] = su_paul.id


In [5]:
food_df.head(2)

,id,name,cals_per_gram,fat_per_gram,carbs_per_gram,protein_per_gram,sugar_per_gram,date,food_type_id,food_group_id,user_id
0,1,"Butter, salted",7.17,7.2999,0.0024,0.0340,0.0024,2022-03-27 13:15:51.408017,1,1,1
1,2,"Butter, whipped, with salt",7.18,7.0470,0.1148,0.0196,0.0024,2022-03-27 13:15:51.450273,1,1,1


In [16]:
m_f_fd_gf = food_df.merge(food_group_df[['id', 'rank']], how='inner', left_on='food_group_id', right_on='id')
m_f_fd_gf = m_f_fd_gf.rename(columns={"id_x": "id", "id_y": "fg_id", 'rank': 'fg_rank'})

m_f_fd_gf['food_group_id'] = m_f_fd_gf['fg_rank']

food_df = m_f_fd_gf[[c for c in m_f_fd_gf.columns if c not in ('fg_id', 'fg_rank',)]]




,id,name,cals_per_gram,fat_per_gram,carbs_per_gram,protein_per_gram,sugar_per_gram,date,food_type_id,food_group_id,user_id
0,1,"Butter, salted",7.17,7.2999,0.0024,0.034,0.0024,2022-03-27 13:15:51.408017,1,8,1


In [29]:
food_group_df.columns = ['old_id', 'name', 'fg_rank']
for old_id, name, rank in food_group_df.itertuples(index=False):
        
        fg = FoodGroup.objects.get(id=rank)
        assert fg.name == name
        assert fg.id == rank
        

In [44]:
for row in food_df[1:].itertuples(index=False):
    id_, name, cals, fat, carbs, prot, sugar, date, food_type_id, food_group_id, user_id = row
    food_type = FoodType.objects.get(id=food_type_id)
    food_group = FoodGroup.objects.get(id=food_group_id)
    user = MacroUser.objects.get(id=user_id)
    Foods.objects.create(
        name=name, cals_per_gram=cals, fat_per_gram=fat, carbs_per_gram=carbs, sugar_per_gram=sugar,
        food_type=food_type, food_group=food_group, user=user
    )
    
print('done')

done


In [3]:
tmp_user = MacroUser.objects.create_user(email='e@e.com', password="password")

In [6]:
su_paul = MacroUser.objects.get(email='paul@email.com')
usda = MacroUser.objects.get(email='usda@email.com')
(su_paul, su_paul.id,), (usda, usda.id)

((<MacroUser: paul@email.com>, 1), (<MacroUser: usda@email.com>, 2))

In [8]:
Foods.objects.filter(user=su_paul).update(user=tmp_user)

4988

In [16]:
tmp_user.delete()

(1,
 {'admin.LogEntry': 0,
  'accounts.MacroUser_groups': 0,
  'accounts.MacroUser_user_permissions': 0,
  'meals.Macros': 0,
  'accounts.MacroUser': 1})

In [63]:
SORT BY OLD ID?

food_group_dict = {}
for idx, df_row in enumerate(food_group_df.itertuples(index=False)):
    new_id = idx + 1
    old_id, name, rank = df_row
    print(new_id, old_id, name, rank)
    fg = !FoodGroup.objects.create(name=name, rank=rank)
    if fg.id != new_id:
        fg.delete()
        raise Exception('reset fg pk increment - id <> new_id')
    food_df.loc[food_df.food_group_id == old_id, 'food_group_id'] = fg.id

    

1 28 My Foods 1
2 26 My Meals 2
3 27 My Recipes 3
4 4 Fats and Oils 4
5 11 Veggies 5
6 9 Fruits 6
7 5 Meat 7
8 1 Dairy and Eggs 8
9 20 Grains and Pasta 9
10 12 Nuts and Seeds 10
11 15 Fish and Seafood 11


In [61]:
[[i, n, g] for i, n, g in food_group_df.itertuples(index=False)]

[[28, 'My Foods', 1],
 [26, 'My Meals', 2],
 [27, 'My Recipes', 3],
 [4, 'Fats and Oils', 4],
 [11, 'Veggies', 5],
 [9, 'Fruits', 6],
 [5, 'Meat', 7],
 [1, 'Dairy and Eggs', 8],
 [20, 'Grains and Pasta', 9],
 [12, 'Nuts and Seeds', 10],
 [15, 'Fish and Seafood', 11]]

In [10]:
su_paul = MacroUser.objects.create_superuser("paul@email.com", "password")
usda = MacroUser.objects.create_superuser("usda@email.com", "password")


/home/vagrant/site/envs/macrobatics/lib/python3.7/site-packages/django/db/models/base.py:316: RuntimeWarning: Model 'accounts.macrouser_groups' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/vagrant/site/envs/macrobatics/lib/python3.7/site-packages/django/db/models/base.py:316: RuntimeWarning: Model 'accounts.macrouser_user_permissions' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/vagrant/site/envs/macrobatics/lib/python3.7/site-packages/django/db/models/base.py:316: RuntimeWarning: Model 'accounts.macrouser' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(

In [12]:
food_df.loc[food_df.user_id == 4]

1

In [15]:
su_paul.id, usda.id


(1, 2)

In [ ]:
foodInterface = namedtuple('foodInterface', 'cals, fat, carbs, protein')
FoodMacros = namedtuple("FoodMacros", "cals, fat, carbs, sugar, protein")
FoodMacrosPerGram = namedtuple("FoodMacrosPerGram", "cals_per_gram, fat_per_gram, carbs_per_gram, sugar_per_gram, protein_per_gram")

class FoodCombo:
    def __init__(self, food, grams=None):
        self.food = food
        if grams:
            self.grams = grams
        else:
            self.grams = food.grams
            
        self.cals = food.cals
        self.fat = food.fat
        self.carbs = food.carbs
        self.sugar = food.sugar
        self.protein = food.protein
        
    @property
    def macros(self):
        cals = self.cals
        fat = self.fat / 9
        carbs = self.carbs / 4
        sugar = self.sugar / 4
        protein = self.protein / 4
        return FoodMacros(cals=cals, fat=fat, carbs=carbs, sugar=sugar, protein=protein)

    @property
    def macros_per_gram(self):
        cals_per_gram = self.cals / self.grams
        fat_per_gram = self.fat / self.grams
        carbs_per_gram = self.carbs / self.grams
        sugar_per_gram = self.sugar / self.grams
        protein_per_gram = self.protein / self.grams
        return FoodMacrosPerGram(
            cals_per_gram=cals_per_gram,
            fat_per_gram=fat_per_gram,
            carbs_per_gram=carbs_per_gram,
            sugar_per_gram=sugar_per_gram,
            protein_per_gram=protein_per_gram
        )

    def __add__(self, otherFoodCombo):
        thisFood = self.food
        otherFood = otherFoodCombo.food
        total_grams = self.grams + otherFoodCombo.grams
        cals = thisFood.cals + otherFood.cals
        fat = thisFood.fat + otherFood.fat
        carbs = thisFood.carbs + otherFood.carbs
        sugar = thisFood.sugar + otherFood.sugar
        protein = thisFood.protein + otherFood.protein
        print(cals, fat, carbs, sugar, protein)
        new_food = FoodMacros(cals=cals, fat=fat, carbs=carbs, sugar=sugar, protein=protein)
        new_food_combo = FoodCombo(new_food, grams=total_grams)
        return new_food_combo 
    
    def __repr__(self):
        food = self.food
        return f"Cals: {food.cals}, Fat: {food.fat}, Carbs: {food.carbs}, Sugar: {self.sugar}, Protein: {food.protein}"